In [ ]:
import Pkg;
Pkg.add("LightGraphs");
Pkg.add("Optim");
Pkg.add("DifferentialEquations")
Pkg.add("JSON")
Pkg.add("Plots")
Pkg.add("Rotations");
Pkg.add("LinearAlgebra");

In [1]:
using LightGraphs;
using Optim;
using Random;
using Rotations;
using LinearAlgebra;
using Printf;

In [2]:
function maxl(L)
    m = -1
    for i = 1:length(L)
        if L[i][1]>m
            m = L[i][1]
        end
        if L[i][2]>m
            m = L[i][2]
        end
    end
    return m
end

function from_edge_list(L)
    g = SimpleGraph()
    add_vertices!(g,maxl(L)+1)
    for i = 1:length(L)
        add_edge!(g,L[i][1]+1,L[i][2]+1)

    end
    return g
end


from_edge_list (generic function with 1 method)

In [18]:

eps = BigFloat(1e-20)
eps1 = BigFloat(1)
unit = BigFloat(1)

function f(x)::BigFloat
    s = BigFloat(0)
    global unit
    global g
    global R
    global R2
    global node0
    global node1
    global node2
    for e in edges(g)
        a = 3*(e.src-1)+1
        b = 3*(e.dst-1)+1
        s = s + ((x[a]-x[b])^2 + (x[a+1]-x[b+1])^2+ (x[a+2]-x[b+2])^2 - unit)^2
 

    end
    for n = 1:nv(g)
        a = 3*(n-1)+1
        s = s + (x[a]^2+x[a+1]^2+x[a+2]^2-R^2)^2
 

    end
#    s = s + eps1*((x[1]-node0[1])^2+(x[2]-node0[2])^2+(x[3]-node0[3])^2)
#    s = s + eps1*((x[4]-node1[1])^2+(x[5]-node1[2])^2+(x[6]-node1[3])^2)
#    s = s + eps1*((x[7]-node2[1])^2+(x[8]-node2[2])^2+(x[9]-node2[3])^2)
    return s
end

function check_eq(x,eps=BigFloat(1e-20))
    global g
    r = true
    for i = 1:nv(g)
        for j = i+1:nv(g)
            a = 3*(i-1)+1
            b = 3*(j-1)+1
            d2 = (x[a]-x[b])^2+(x[a+1]-x[b+1])^2+(x[a+2]-x[b+2])^2
            if d2<eps
                r = false
            end
        end
    end
    return r
end


check_eq (generic function with 2 methods)

In [19]:
function triang(x,y)
    x = convert(Array{BigFloat,1},x[:])
    y = convert(Array{BigFloat,1},y[:])
    p = x*(unit/norm(x))
    r = RotationVec(p[1]*(2*pi/5), p[2]*(2*pi/5), p[3]*(2*pi/5))
    z1 = r*y
    r = RotationVec(p[1]*(-2*pi/5), p[2]*(-2*pi/5), p[3]*(-2*pi/5))
    z2 = r*y
    return z1,z2
end



function rot_matr(x1,y1,x2,y2)
    z11,z12 = triang(x1,y1)
    z21,z22 = triang(x2,y2)
    A1 = vcat(x1', y1', z11')
    A2 = vcat(x1', y1', z12')
    B1 = vcat(x2', y2', z21')
    B2 = vcat(y2', x2', z22')
    return [A1^-1*B1,A1^-1*B2,A2^-1*B1,A2^-1*B2]
end


rot_matr (generic function with 1 method)

In [45]:
# 17-vertex Exoo graph
L = [(1,8),(1,12),(1,15),(2,9),(2,10),(2,16),(3,9),(3,15),(3,0),(3,16),(4,11),(4,10),(4,16),(5,11),(5,13),(5,16),(6,12),(6,13),(6,14),(7,12),(7,14),(7,15),(8,13),(8,14),(8,0),(9,11),(9,13),(10,15),(10,0),(11,14),(12,0)]

#10-vertex graph
#L = [(1, 2), (1, 3), (1, 5), (1, 6), (2, 3), (2, 5), (2, 9), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8), (4, 0), (5, 9), (6, 0), (7, 8), (9, 0)]

# triangle
#L = [(0,1),(1,2),(0,2)]

m = length(L)
n = 0
for i = 1:m
    n1 = maximum(L[i])+1 
    if n1>n
        n = n1
    end
end    

R = BigFloat(57)/BigFloat(100)
R2 = R*R

g = from_edge_list(L)


{17, 31} undirected simple Int64 graph

In [ ]:
setprecision(100)

m = 100.0
mx = []
embed =[]


tol = BigFloat(10)^(-20)


using Random;


for i = 1:50
    x0 = unit*rand(n*3) .- unit/2;
    result = optimize(f, x0, BFGS(), Optim.Options(x_tol=tol, f_tol=tol, g_tol=tol))
    m1 = result.minimum
    if check_eq(result.minimizer) & (m1<1e-20)
        mx = result.minimizer
        print("(emb) ")
        push!(embed, mx)
        break
    end
    @printf "precision: %.20f \n" m1
    
end

embed

precision: 0.00064313163709792223 
precision: 0.00019623311656700651 
